In [1]:
import os

from dotenv import load_dotenv

from langchain.llms import OpenAI, HuggingFaceHub

# LLMs at brief

## Get API Key details

In [2]:
load_dotenv('../.env')

True

## Init LLM

### Create & Configure LLM

In [3]:
llm = OpenAI(model='text-davinci-003')

In [4]:
llm('What is your name ?')

'\n\nMy name is Amy.'

In [5]:
hfh_llm = HuggingFaceHub(repo_id='google/flan-t5-small')

/Users/chiraj/iWork/LLM/hello-world-llm/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/chiraj/iWork/LLM/hello-world-llm/.venv/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
hfh_llm('What is your name ?')

'john'

## Trying out Chat models

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [8]:
chat = ChatOpenAI(
    temperature=0.7,
    model='gpt-3.5-turbo'
)

In [9]:
chat([
    SystemMessage(content='You are a sarcastic doctor'),
    HumanMessage(content='Please answer in 50 words, what I should I do for my headache')
])

AIMessage(content='Oh, just take a rocket to the moon and back, and your headache will magically disappear. But seriously, try drinking plenty of water, taking a break from screens, and maybe popping a painkiller. If it persists, maybe consider seeing an actual doctor.', additional_kwargs={}, example=False)

In [10]:
chat([
    SystemMessage(content='Answer like a Sarcastic Software engineer'),
    HumanMessage(content='Please answer in 50 words, what I should I do for my headache'),
    AIMessage(content='Try to reboot yourself'),
    HumanMessage(content='Tell me about yourself')
])

AIMessage(content="Oh, well, I'm just your friendly neighborhood AI programmed to assist with all your questions and tasks. I'm here to make your life easier, or at least attempt to. So, how can I sarcastically help you today?", additional_kwargs={}, example=False)

## Embeddings

In [11]:
from langchain.embeddings import OpenAIEmbeddings
from openai.embeddings_utils import cosine_similarity

In [12]:
embeddings = OpenAIEmbeddings()

In [13]:
samp_txt = 'Hello World'

samp_txt_embed = embeddings.embed_query(samp_txt)

len(samp_txt_embed), samp_txt_embed

(1536,
 [-0.007058123716785191,
  0.0034781466818117803,
  -0.00699447992068217,
  -0.029047139555079107,
  -0.012970655378425388,
  0.01087676638412891,
  -0.020276989620615565,
  0.005059700882304571,
  -0.008451928159980478,
  -0.03024364701963558,
  0.024324750698988283,
  0.009661164942551145,
  -0.0274687674512592,
  -0.0066380729861244755,
  0.009120190347368832,
  0.015567331945184244,
  0.02207175268009558,
  -0.009546605737036645,
  0.010959503225930572,
  0.015567331945184244,
  -0.0056515904212370354,
  0.00699447992068217,
  0.005397014305502297,
  0.005075612064160989,
  -0.013925315113938665,
  -0.006068458822394202,
  0.005409743157855167,
  -0.025457615298764388,
  0.030752799251105054,
  -0.02998906997257819,
  0.009769359489058545,
  -0.01230239225971438,
  -0.00249166411692434,
  -0.028054291865523242,
  0.007751843608879285,
  -0.012563333034456217,
  -0.0026253166009681437,
  -0.014879975780774594,
  0.018329482195546424,
  -0.01937324529451377,
  0.00500560360905

In [14]:
sample_txts = ['Apple Inc', 'Nokia', 'Samsung', 'Carrot', 'Tesla', 'NASA', 'Crime Investigation']

sample_txts_embed = list(map(lambda x: embeddings.embed_query(x), sample_txts))

In [15]:
for i, txt_embed in enumerate(sample_txts_embed):
    print(sample_txts[0], sample_txts[i], '----', cosine_similarity(sample_txts_embed[0], txt_embed))

Apple Inc Apple Inc ---- 1.0
Apple Inc Nokia ---- 0.8396381824731832
Apple Inc Samsung ---- 0.813859778766305
Apple Inc Carrot ---- 0.7875628025138957
Apple Inc Tesla ---- 0.8143827624947114
Apple Inc NASA ---- 0.7879536876716569
Apple Inc Crime Investigation ---- 0.7672331447387873


# Prompts

## Prompt Template

In [16]:
from langchain import PromptTemplate

In [17]:
llm = OpenAI(model='text-davinci-003') 

In [18]:
template = "Tell me a {adjective} joke about {content}."
prompt_template = PromptTemplate(
    input_variables=['adjective', 'content'],
    template=template
)

prompt = prompt_template.format(adjective="funny", content="chickens")

prompt

'Tell me a funny joke about chickens.'

In [19]:
llm(prompt)

'\n\nQ: Why did the chicken go to the séance?\nA: To get to the other side.'

## Example Selectors

In [20]:
from langchain import FewShotPromptTemplate

In [21]:
examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
  },
  {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

In [22]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



In [23]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Question: {input}", 
    input_variables=["input"]
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

In [24]:
print(llm(prompt.format(input="Who was the maternal father of Mr. Bean?")))



Are follow up questions needed here: Yes.
Follow up: Who is Mr. Bean?
Intermediate Answer: Mr. Bean is a fictional character portrayed by British comedian Rowan Atkinson.
Follow up: Who is the mother of Rowan Atkinson?
Intermediate Answer: The mother of Rowan Atkinson is Ella May Bainbridge.
Follow up: Who is the father of Ella May Bainbridge?
Intermediate Answer: The father of Ella May Bainbridge is Eric Atkinson.
So the final answer is: Eric Atkinson


## Output Parsers

In [25]:
from langchain.output_parsers import CommaSeparatedListOutputParser, StructuredOutputParser, ResponseSchema

### List output

In [26]:
output_parser = CommaSeparatedListOutputParser()

In [27]:
format_instructions = output_parser.get_format_instructions()

format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [28]:
template = "Tell me 5 {adjective} joke about {content}.\n{format_instructions}"

prompt_template = PromptTemplate(
    input_variables=['adjective', 'content'],
    template=template,
    partial_variables={"format_instructions": format_instructions}
)

prompt = prompt_template.format(adjective="funny", content="chickens")

prompt

'Tell me 5 funny joke about chickens.\nYour response should be a list of comma separated values, eg: `foo, bar, baz`'

In [29]:
print(llm(prompt))



"Why did the chicken go to the séance? To get to the other side!", "Why don't chickens like people? Because they tend to get picked on!", "What's a chicken's favorite city? Poul-try!", "Why did the chicken cross the playground? To get to the other slide!", "Why did the chicken cross the basketball court? He heard the referee call fowl!"


### JSON output

In [30]:
resp_schema = [
    ResponseSchema(name='currency', description='Answer to the question'),
    ResponseSchema(name='abbrv', description='Abbrevation of the currency'),
    ResponseSchema(name='value', description='Value of the currency')
]

In [31]:
output_parser = StructuredOutputParser(response_schemas=resp_schema)

In [32]:
template = "Tell me values of {num} different currencies in USD. \n {format_instructions}"

prompt_template = PromptTemplate(
    input_variables=['num'],
    template=template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

prompt = prompt_template.format(num="5")

prompt

'Tell me values of 5 different currencies in USD. \n The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"currency": string  // Answer to the question\n\t"abbrv": string  // Abbrevation of the currency\n\t"value": string  // Value of the currency\n}\n```'

In [33]:
print(llm(prompt))



```json
{
	"currency": "Euro",
	"abbrv": "EUR",
	"value": "1.14"
}
{
	"currency": "British Pound",
	"abbrv": "GBP",
	"value": "0.76"
}
{
	"currency": "Indian Rupee",
	"abbrv": "INR",
	"value": "71.11"
}
{
	"currency": "Japanese Yen",
	"abbrv": "JPY",
	"value": "108.74"
}
{
	"currency": "Australian Dollar",
	"abbrv": "AUD",
	"value": "1.43"
}
```


# Memory

In [34]:
from langchain.memory import ConversationBufferWindowMemory, ConversationTokenBufferMemory
from langchain.chains import ConversationChain

In [35]:
convo = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=3)
)

In [36]:
convo('I\'m CJ')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I'm CJ
AI:

> Finished chain.


{'input': "I'm CJ",
 'history': '',
 'response': ' Nice to meet you, CJ! My name is AI. What would you like to talk about?'}

In [37]:
convo('Who\'re you ?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm CJ
AI:  Nice to meet you, CJ! My name is AI. What would you like to talk about?
Human: Who're you ?
AI:

> Finished chain.


{'input': "Who're you ?",
 'history': "Human: I'm CJ\nAI:  Nice to meet you, CJ! My name is AI. What would you like to talk about?",
 'response': ' I am an Artificial Intelligence developed to interact with humans like you. I have been designed to answer questions and have conversations with you. What else would you like to know?'}

In [38]:
convo('who am I')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm CJ
AI:  Nice to meet you, CJ! My name is AI. What would you like to talk about?
Human: Who're you ?
AI:  I am an Artificial Intelligence developed to interact with humans like you. I have been designed to answer questions and have conversations with you. What else would you like to know?
Human: who am I
AI:

> Finished chain.


{'input': 'who am I',
 'history': "Human: I'm CJ\nAI:  Nice to meet you, CJ! My name is AI. What would you like to talk about?\nHuman: Who're you ?\nAI:  I am an Artificial Intelligence developed to interact with humans like you. I have been designed to answer questions and have conversations with you. What else would you like to know?",
 'response': " I'm sorry, I don't know who you are. Can you tell me a bit about yourself?"}

In [39]:
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [40]:
convo.predict(input='I\'m CJ')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm CJ
AI:  Nice to meet you, CJ! My name is AI. What would you like to talk about?
Human: Who're you ?
AI:  I am an Artificial Intelligence developed to interact with humans like you. I have been designed to answer questions and have conversations with you. What else would you like to know?
Human: who am I
AI:  I'm sorry, I don't know who you are. Can you tell me a bit about yourself?
Human: I'm CJ
AI:

> Finished chain.


'  Nice to meet you, CJ! What would you like to talk about?'

In [41]:
print(convo.memory.buffer)

Human: Who're you ?
AI:  I am an Artificial Intelligence developed to interact with humans like you. I have been designed to answer questions and have conversations with you. What else would you like to know?
Human: who am I
AI:  I'm sorry, I don't know who you are. Can you tell me a bit about yourself?
Human: I'm CJ
AI:   Nice to meet you, CJ! What would you like to talk about?


In [42]:
convo_token = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationTokenBufferMemory(max_token_limit=30, llm=llm)
)

In [43]:
convo_token('I\'m CJ')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I'm CJ
AI:

> Finished chain.


{'input': "I'm CJ",
 'history': '',
 'response': " Hi CJ, I'm AI. It's nice to meet you. How can I help you today?"}

In [44]:
convo_token('Hello World')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm CJ
AI:  Hi CJ, I'm AI. It's nice to meet you. How can I help you today?
Human: Hello World
AI:

> Finished chain.


{'input': 'Hello World',
 'history': "Human: I'm CJ\nAI:  Hi CJ, I'm AI. It's nice to meet you. How can I help you today?",
 'response': " Hi there! It's nice to meet you. I'm AI, what can I do for you today?"}

In [45]:
convo_token.memory.buffer

"Human: Hello World\nAI:  Hi there! It's nice to meet you. I'm AI, what can I do for you today?"